In [18]:
import string
import re
import os
import tempfile
import logging
import csv
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from datetime import datetime
from gensim import corpora
from gensim import models
from gensim.corpora import Dictionary
from collections import namedtuple
import gensim.parsing.preprocessing as processing
from os import listdir
import CustomApi as api
#Start Global Variables and Types

#document dir
docs = "../data/documents/"

#Number of topics the model should index
numberOfTopics = 225

#Number of passes the model should make
passes = 75

#Keyphrase tracker
keyphraseTracker = api.KeyWordTracker()

#End Global Variables and Types

In [19]:
files = listdir(docs+"gt2006/")
entries = []

In [20]:
for file in files:
    with open(docs+"gt2006/"+file,"rb") as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if "Authors" not in row[0] and row[15] !='[No abstract available]':
                entries.append(row)
        

In [21]:
#title of an abstract
entries[0][1]


'Proteomic analysis of honeybee worker (Apis mellifera) hypopharyngeal gland development'

In [22]:
# Create a set of frequent words
stopFile = open("../data/stopwords.txt","r")
stopWords = stopFile.read().splitlines() #This is the copyright symbol, this shows up in every abstract and should not be apart of the corpus
stopWords.extend([u"\u2019",u"\u03bc","bee","bees","honey","honeybee","honeybees",u"\xa9",u"\xc2"])
# for asc in range(97,123):
#     stopWords.extend([chr(asc)])
with open("../data/extraStopWords.txt","r") as extraStopFile:
    stopWords.extend(extraStopFile.read().split("\n"))
# Lowercase each document, split it by white space and filter out stopWords
stopList = set(stopWords)
processing.STOPWORDS = stopList
ps = PorterStemmer()
def removeStops(text):
    stopsRemoved = processing.remove_stopwords(text.lower().translate(None, string.punctuation))
    words = stopsRemoved.split(" ")
    stemmedWords = []
    for w in words:
        if len(ps.stem(w)) > 2:
            stemmedWords.append(ps.stem(w))
    return ' '.join(stemmedWords)
# Each abstract has a 'title':String, 'date':datetime.datetime, 'text':String, and 'keywords':String
abstracts = [api.MyAbstract._make([art[1],datetime.strptime(art[2], '%Y'),removeStops(art[15]), art[16]]) for art in entries]
abstracts.sort(key=lambda q: q.date.year)
entries = None
# Count word frequencies
selections = []

access = lambda x: x.date.year
lastIndex = 0
for i in range(2007,2019):
    index = api.binarySearch(abstracts,i,access)
    if  index != -1:
        selections.append(abstracts[lastIndex:index+1])
        lastIndex = index+1
        

In [23]:
from collections import defaultdict
def createCorpus(selection):
    frequency = defaultdict(int)
    for abst in abstracts:
        for token in abst.text.split(" "):
            frequency[token] += 1
    tempFolder = tempfile.gettempdir()
    processedCorpus = [[token for token in abst.text.split(" ") if frequency[token] > 5] for abst in selection]
    dictionary = corpora.Dictionary(processedCorpus)
    dictionary.save(os.path.join(tempFolder,'words.dict'))
    # Create general corpus and serialize in order for it to be iterated over
    corpus = [dictionary.doc2bow(text) for text in processedCorpus]
    corpora.MmCorpus.serialize(os.path.join(tempFolder, 'words.dict'), corpus)
    return api.MyCorpora._make([corpus,dictionary])
    


# Save the dictionary of tokens
# def createModel():
#     tempFolder = tempfile.gettempdir()
#     dictionary = corpora.Dictionary(processedCorpusFor(selection))
#     dictionary.save(os.path.join(tempFolder,'words.dict'))
#     # Create general corpus and serialize in order for it to be iterated over
#     corpus = [dictionary.doc2bow(text) for text in processedCorpus]
#     corpora.MmCorpus.serialize(os.path.join(tempFolder, 'words.dict'), corpus)
    # Train the model and set number of topics


In [24]:
numTopicsExport = 50

def exportResults(path, sortedVals,wordDict):
    seg = sortedVals[0][0:numTopicsExport]
    #Edges and their weights [(a,b):weight]
    edges = sortedVals[1]
    # set of (a,b)
    exportEdges = set({})
    exportNodes = set({})
    for node in seg:
        exportNodes.add((node["tag"],node["word"],node["occurences"]))
        for e in node["edges"]:
            a = wordDict[e[0]]
            b = wordDict[e[1]]
            if a in seg and b in seg:
                exportEdges.add(e)
                exportNodes.add((a["tag"],a["word"],a["occurences"]))
                exportNodes.add((b["tag"],b["word"],b["occurences"]))
            
    with open(path+"nodes.csv","w") as file:
        file.write("Id,Label,Weight\n")
        for val in exportNodes:
            file.write(str(val[0])+","+val[1].encode('utf8')+','+str(val[2])+'\n')
    with open(path+'edges.csv',"w") as file:
        file.write("Source,Target,Weight\n")
        #We dont want to have duplicate edges written, so we want to make an aedge buffer and then write the buffer
        for e in exportEdges:
            file.write(str(e[0])+","+str(e[1])+","+str(edges[e])+"\n")
#         edgesToWrite = set([])
#         for val in seg:
#             #Edges a node has
#             for edge in val["edges"]:
#                 #Check if (a,b) is an edge else if (b,a) is an edge. The graph is undirected so we dont want duplicates
#                 if edge in edges.keys():
#                     #Edge wasnt in the buffer, add it
#                     if edge not in edgesToWrite:
#                         edgesToWrite.add(edge)
#                 #(b,a) in edge list?
#                 elif (edge[1],edge[0]) in edges.keys():
#                     #edge wasnt in the buffer
#                     edgesToWrite.add(edge)
#         for edge in edgesToWrite:
#             file.write(str(edge[0])+","+str(edge[1])+","+str(edges[edge])+"\n")

In [25]:
import math as m
def getResults(topics):
    mapper = api.WordMapper()
    #Edges [(a,b):weight]
    edges = {}
    def updateEdgeFreq(vec1):
        if vec1[0] == vec1[1]:
            return None
        altVec = [vec1[1],vec1[0]]
        if vec1 in edges.keys():
            edges[vec1] += 1
            return vec1
        elif altVec in edges.keys():
            edges[altVec] += 1
            return altVec
        else:
            edges[vec1] = 1
            return vec1
    occurences = {}
    topics[len(topics)-1]
    for topic in topics:
        words = topic[0]
        #make all the edges for this topic
        wordEdges = []
        wid = mapper.mapWord(words[0][1])
        for t in words:
            edge = updateEdgeFreq((wid,mapper.mapWord(t[1])))
            if edge != None:
                wordEdges.append(edge)
        #add the edges to each word and and update the number of occurences per word
        for w in words:
            word = w[1]
            wid = mapper.mapWord(word)
            if wid not in occurences.keys():
                occurences[wid] = {"prob":w[0],"occurences":1,"edges":wordEdges, "tag" : wid, "word":word}
            else:
                occurences[wid]["occurences"] += 1
                occurences[wid]["prob"] = max(occurences[wid]["prob"], w[0])
    return [occurences,edges]


In [26]:
from difflib import SequenceMatcher
size = 0
start = 0
import os
occurences = None
for i in range(0,len(selections)):
    keyphraseTracker = api.KeyWordTracker()
    size += len(selections[i])
    if size >=50:
        selection = api.flaten(selections[start:i+1])
       
        corpus = createCorpus(selection)
        path = docs+'GephiFiles/'+str(selection[0].date.year)+'/'
        try:
            os.makedirs(path)
            print "made dirs"
        except:
            #Already exists
            pass
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        lda = models.ldamodel.LdaModel(corpus.corpus,id2word=corpus.id2word,num_topics=numberOfTopics,passes=passes)
        topics = lda.top_topics(corpus=corpus.corpus,dictionary=corpus.id2word,topn=3)
        
        topics.sort(key=lambda k:k[1],reverse=True)
        occurences = getResults(topics)
        sortedVals = [sorted(occurences[0].values(),key=lambda k:k["occurences"],reverse=True),occurences[1]]
        #####Release some of this memory please and thank you
        selection = None
        #occurences = None
        #####
        exportResults(path,sortedVals,occurences[0])
        size = 0
        start = i+1
#for y in keyphraseTracker.topN():
    #print str(y.getPhrase())+", "+str(y.sum())

2018-06-27 10:15:49,388 : INFO : using symmetric alpha at 0.00444444444444
2018-06-27 10:15:49,392 : INFO : using symmetric eta at 0.00444444444444
2018-06-27 10:15:49,397 : INFO : using serial LDA version on this node


made dirs


2018-06-27 10:15:49,641 : INFO : running online (multi-pass) LDA training, 225 topics, 75 passes over the supplied corpus of 533 documents, updating model once every 533 documents, evaluating perplexity every 533 documents, iterating 50x with a convergence threshold of 0.001000
2018-06-27 10:15:53,186 : INFO : -106.332 per-word bound, 102128728026182095922094665629696.0 perplexity estimate based on a held-out corpus of 533 documents with 46462 words
2018-06-27 10:15:53,190 : INFO : PROGRESS: pass 0, at document #533/533
2018-06-27 10:15:56,358 : INFO : topic #11 (0.004): 0.029*"pollen" + 0.028*"winter" + 0.021*"fall" + 0.019*"mate" + 0.018*"type" + 0.018*"contain" + 0.018*"gene" + 0.017*"transcript" + 0.017*"protein" + 0.014*"singl"
2018-06-27 10:15:56,361 : INFO : topic #139 (0.004): 0.038*"mite" + 0.027*"ant" + 0.022*"varroa" + 0.015*"forag" + 0.012*"formica" + 0.012*"condit" + 0.011*"respons" + 0.011*"trial" + 0.011*"pollin" + 0.011*"nonassoci"
2018-06-27 10:15:56,365 : INFO : topic

2018-06-27 10:16:10,849 : INFO : topic #97 (0.004): 0.017*"condit" + 0.014*"genet" + 0.014*"tribe" + 0.011*"coloni" + 0.011*"analys" + 0.011*"per" + 0.011*"pcrrflp" + 0.011*"support" + 0.011*"cleptoparasit" + 0.011*"monophyli"
2018-06-27 10:16:10,852 : INFO : topic #171 (0.004): 0.031*"numt" + 0.027*"mitochondri" + 0.023*"sequenc" + 0.022*"pure" + 0.022*"strain" + 0.022*"gene" + 0.022*"origin" + 0.018*"day" + 0.018*"locat" + 0.018*"nuclear"
2018-06-27 10:16:10,859 : INFO : topic diff=inf, rho=0.377964
2018-06-27 10:16:12,446 : INFO : -8.951 per-word bound, 494.8 perplexity estimate based on a held-out corpus of 533 documents with 46462 words
2018-06-27 10:16:12,447 : INFO : PROGRESS: pass 6, at document #533/533
2018-06-27 10:16:13,780 : INFO : topic #208 (0.004): 0.003*"incid" + 0.002*"year" + 0.001*"poison" + 0.001*"pesticid" + 0.001*"fumagillin" + 0.001*"mite" + 0.001*"per" + 0.001*"gene" + 0.001*"efficaci" + 0.001*"thymol"
2018-06-27 10:16:13,782 : INFO : topic #111 (0.004): 0.044*

2018-06-27 10:16:27,897 : INFO : topic #98 (0.004): 0.038*"antinocicept" + 0.036*"pheromon" + 0.028*"korean" + 0.024*"inject" + 0.021*"queen" + 0.021*"efficaci" + 0.021*"min" + 0.021*"mgkg" + 0.017*"rat" + 0.017*"formalin"
2018-06-27 10:16:27,898 : INFO : topic #155 (0.004): 0.040*"honeybeecollect" + 0.040*"phenol" + 0.039*"pollen" + 0.027*"determin" + 0.027*"peak" + 0.027*"origin" + 0.013*"american" + 0.013*"optimum" + 0.013*"therapeut" + 0.013*"rang"
2018-06-27 10:16:27,907 : INFO : topic diff=inf, rho=0.277350
2018-06-27 10:16:29,494 : INFO : -8.341 per-word bound, 324.2 perplexity estimate based on a held-out corpus of 533 documents with 46462 words
2018-06-27 10:16:29,499 : INFO : PROGRESS: pass 12, at document #533/533
2018-06-27 10:16:30,641 : INFO : topic #56 (0.004): 0.025*"gene" + 0.022*"worker" + 0.018*"reproduct" + 0.015*"queen" + 0.013*"scout" + 0.013*"protein" + 0.011*"differ" + 0.011*"swarm" + 0.011*"site" + 0.011*"odor"
2018-06-27 10:16:30,644 : INFO : topic #17 (0.004)

...

2018-06-27 11:13:57,391 : INFO : topic diff=inf, rho=0.142857
2018-06-27 11:13:58,880 : INFO : -8.261 per-word bound, 306.7 perplexity estimate based on a held-out corpus of 339 documents with 33276 words
2018-06-27 11:13:58,881 : INFO : PROGRESS: pass 48, at document #339/339
2018-06-27 11:13:59,903 : INFO : topic #115 (0.004): 0.000*"endophyt" + 0.000*"leaf" + 0.000*"resin" + 0.000*"sex" + 0.000*"bud" + 0.000*"fungi" + 0.000*"male" + 0.000*"damag" + 0.000*"rich" + 0.000*"dracunculifolia"
2018-06-27 11:13:59,905 : INFO : topic #134 (0.004): 0.081*"eleph" + 0.036*"asian" + 0.036*"sound" + 0.027*"protect" + 0.018*"african" + 0.018*"conduct" + 0.018*"trial" + 0.018*"control" + 0.018*"respond" + 0.018*"disturb"
2018-06-27 11:13:59,911 : INFO : topic #126 (0.004): 0.050*"rear" + 0.022*"environ" + 0.019*"behavior" + 0.015*"heat" + 0.014*"compar" + 0.014*"artifici" + 0.014*"stress" + 0.012*"affect" + 0.011*"introduc" + 0.011*"vitro"
2018-06-27 11:13:59,917 : INFO : topic #160 (0.004): 0.000*

2018-06-27 11:14:11,871 : INFO : topic diff=inf, rho=0.134840
2018-06-27 11:14:13,102 : INFO : -8.260 per-word bound, 306.7 perplexity estimate based on a held-out corpus of 339 documents with 33276 words
2018-06-27 11:14:13,108 : INFO : PROGRESS: pass 54, at document #339/339
2018-06-27 11:14:14,272 : INFO : topic #170 (0.004): 0.000*"endophyt" + 0.000*"sex" + 0.000*"male" + 0.000*"resin" + 0.000*"bud" + 0.000*"leaf" + 0.000*"fungi" + 0.000*"dracunculifolia" + 0.000*"damag" + 0.000*"rich"
2018-06-27 11:14:14,276 : INFO : topic #118 (0.004): 0.031*"nosema" + 0.031*"infect" + 0.029*"cerana" + 0.019*"nectar" + 0.018*"pollen" + 0.016*"bulgaria" + 0.016*"adhes" + 0.016*"protein" + 0.016*"day" + 0.013*"melliferum"
2018-06-27 11:14:14,279 : INFO : topic #10 (0.004): 0.043*"strain" + 0.036*"fruit" + 0.036*"longev" + 0.030*"reduc" + 0.024*"hour" + 0.018*"spray" + 0.018*"test" + 0.018*"tree" + 0.018*"147" + 0.018*"contact"
2018-06-27 11:14:14,286 : INFO : topic #155 (0.004): 0.047*"brood" + 0.0

2018-06-27 11:14:25,813 : INFO : topic diff=inf, rho=0.128037
2018-06-27 11:14:27,072 : INFO : -8.260 per-word bound, 306.6 perplexity estimate based on a held-out corpus of 339 documents with 33276 words
2018-06-27 11:14:27,075 : INFO : PROGRESS: pass 60, at document #339/339
2018-06-27 11:14:28,059 : INFO : topic #126 (0.004): 0.050*"rear" + 0.022*"environ" + 0.019*"behavior" + 0.015*"heat" + 0.014*"compar" + 0.014*"artifici" + 0.014*"stress" + 0.012*"affect" + 0.011*"introduc" + 0.011*"vitro"
2018-06-27 11:14:28,062 : INFO : topic #118 (0.004): 0.031*"nosema" + 0.031*"infect" + 0.029*"cerana" + 0.020*"nectar" + 0.018*"pollen" + 0.016*"bulgaria" + 0.016*"adhes" + 0.016*"protein" + 0.016*"day" + 0.013*"melliferum"
2018-06-27 11:14:28,066 : INFO : topic #172 (0.004): 0.017*"applic" + 0.017*"odor" + 0.016*"coloni" + 0.014*"extract" + 0.014*"antioxid" + 0.014*"concentr" + 0.014*"reduc" + 0.011*"malt" + 0.011*"juli" + 0.011*"neuron"
2018-06-27 11:14:28,069 : INFO : topic #19 (0.004): 0.05

2018-06-27 11:14:40,901 : INFO : topic diff=inf, rho=0.122169
2018-06-27 11:14:42,305 : INFO : -8.260 per-word bound, 306.6 perplexity estimate based on a held-out corpus of 339 documents with 33276 words
2018-06-27 11:14:42,309 : INFO : PROGRESS: pass 66, at document #339/339
2018-06-27 11:14:43,377 : INFO : topic #103 (0.004): 0.023*"bread" + 0.019*"inhibit" + 0.015*"drug" + 0.015*"inflammatori" + 0.015*"commun" + 0.015*"propoli" + 0.015*"bacteri" + 0.015*"microbiom" + 0.011*"environ" + 0.011*"distinct"
2018-06-27 11:14:43,378 : INFO : topic #157 (0.004): 0.049*"visual" + 0.025*"condit" + 0.021*"blueberri" + 0.020*"test" + 0.018*"fruit" + 0.015*"perform" + 0.014*"combin" + 0.014*"per" + 0.014*"model" + 0.014*"set"
2018-06-27 11:14:43,381 : INFO : topic #212 (0.004): 0.030*"area" + 0.027*"gilliamella" + 0.027*"insecticid" + 0.024*"strain" + 0.020*"dorsata" + 0.020*"insecticidetr" + 0.020*"dna" + 0.016*"lineag" + 0.016*"isol" + 0.016*"cell"
2018-06-27 11:14:43,385 : INFO : topic #11 (0

2018-06-27 11:14:55,463 : INFO : topic diff=inf, rho=0.117041
2018-06-27 11:14:56,811 : INFO : -8.260 per-word bound, 306.5 perplexity estimate based on a held-out corpus of 339 documents with 33276 words
2018-06-27 11:14:56,815 : INFO : PROGRESS: pass 72, at document #339/339
2018-06-27 11:14:57,861 : INFO : topic #53 (0.004): 0.075*"ige" + 0.066*"reactiv" + 0.056*"reaction" + 0.047*"api" + 0.038*"patient" + 0.028*"sensit" + 0.028*"local" + 0.028*"extract" + 0.027*"relationship" + 0.019*"germanica"
2018-06-27 11:14:57,865 : INFO : topic #141 (0.004): 0.058*"parasit" + 0.046*"protein" + 0.031*"cast" + 0.031*"drone" + 0.023*"worker" + 0.023*"abund" + 0.023*"pupa" + 0.022*"varroa" + 0.015*"coloni" + 0.015*"metabol"
2018-06-27 11:14:57,868 : INFO : topic #145 (0.004): 0.069*"spermatozoa" + 0.058*"spermatheca" + 0.046*"movement" + 0.035*"insemin" + 0.035*"insid" + 0.023*"queen" + 0.023*"visibl" + 0.023*"stain" + 0.023*"chang" + 0.023*"circl"
2018-06-27 11:14:57,872 : INFO : topic #111 (0.0

The above corpus shows the amount of times every word used in the documents is used in every indevidual document. Every word is represented by a token ID, the list of which can be found in "words.dict"

In [10]:
# Sort the most interesting words per topic per document
# This cell does not need to be run if only trying to create Top Nine terms per paper
topicOrganizingFile = open("../data/topicorganization.tsv","w")
for x in xrange(0,len(abstracts)):
    doc = dictionary.doc2bow(abstracts[x].split())
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topicOrganizingFile.write(yearOfAbstract[x]+"\t"+titleOfAbstract[x]+"\t")
    for y in xrange(0,min(3,len(docTopics))):
        topicnumber = docTopics[y][0]
        topicOrganizingFile.write(str(lda.show_topic(topicnumber))+"\t")
        #Sorts the word topics in decending order based on their greatest phi value
        for z in xrange(0,len(phiValues)):
            phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
        phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
        curindex=0
        topwords = ""
        for z in xrange(0,3):
            while curindex<len(phiValues) and phiValues[curindex][1][0][0]!=topicnumber:
                curindex+=1
            if(curindex>=len(phiValues)):break
            print len(phiValues)
            print dictionary[phiValues[curindex][0]]
            topwords+=str(dictionary[phiValues[curindex][0]].encode('utf-8').strip())+" "
            curindex+=1
        filter(lambda a:a[0]!=topicnumber,phiValues)
        topicOrganizingFile.write(topwords+"\t")
    topicOrganizingFile.write("\n")
topicOrganizingFile.close()
        

NameError: name 'dictionary' is not defined

In [ ]:
topicWords = []
for i in range(0,numberOfTopics):
    t = lda.get_topic_terms(i,50)
    currentWordList = []
    for x in t:
        word = str(dictionary[x[0]])
        if word not in currentWordList:
            currentWordList.append(word)
    topicWords.append(currentWordList)
topicListFile = open("../data/TopicWords/List-"+str(numberOfTopics)+".txt","w+")
for i in range(0,len(topicWords)):
    topicListFile.write("Topic "+str(i)+":\n")
    for j in topicWords[i]:
        topicListFile.write(j+'\n')
    topicListFile.write('\n')
topicListFile.close()

In [ ]:
#Makes the top nine terms for each document

topNineFile = open("../data/Docbow/TopNineTerms-"+str(numberOfTopics)+".tsv","w")
for abstr in abstracts:
    doc = dictionary.doc2bow(abstracts[2].split()) # Convert to bag of words format first
    # Get the topics and words associated with each document
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topNineFile.write(yearOfAbstract[x]+"\t"+abst+"\t")
    for z in xrange(0,len(phiValues)):
        phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
    phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
    nineWords = ""
    for x in phiValues[:15]:
        nineWords+= dictionary[x[0]] + " "
    topNineFile.write(nineWords.encode('utf-8')+"\n")

In [ ]:
first = api.flaten(yearlyCorpora[0])
first.sort(key=lambda x: x[0])
print api.binarySearch(first,0,lambda x: x[0])
print first

In [ ]:
vals = occurences.values()
vals.sort(key=lambda x:x['occurences'],reverse=True)
print vals[0]

In [ ]:
sortedVals = sorted(occurences.values(),key=lambda k:k["tag"])


In [38]:
y = keyphraseTracker.words[ps.stem("colony")]
print y.years

{1985: 1, 2001: 1, 2004: 1, 2005: 1, 1999: 1}


In [ ]:
 for abstr in selection:
            phrases = [k.strip() for k in  abstr.keywords.split(";")]
            for phr in phrases:
                words = [ps.stem(w) if all(ord(c) < 128 for c in w) else None for w in phr.split(" ")]
                string = ""
                for w in words:
                    if w != None:
                        string = string +" "+ w
                string = string.strip()
                if len(string) > 2:
                    keyphraseTracker.track(string,abstr.date.year)
        #continue to next section